In [1]:
# Import libraries
%matplotlib inline
import math
import numpy as np
import pandas as pd
import seaborn as sns
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
sns.set()

import warnings

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
#from keras.optimizers import SGD, Adam, Adadelta, RMSprop
import keras.backend as K

# Train-Test
from sklearn.model_selection import train_test_split

# Scale
from sklearn.preprocessing import StandardScaler

# Classification Report
from sklearn.metrics import classification_report

In [4]:
df=pd.read_csv('data/players_19.csv')

In [5]:
df.head(30)

,sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club,...,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
0,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,33,1985-02-05,187,83,Portugal,Juventus,...,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3
1,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,31,1987-06-24,170,72,Argentina,FC Barcelona,...,63+2,61+2,61+2,61+2,63+2,58+2,47+2,47+2,47+2,58+2
2,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Junior,26,1992-02-05,175,68,Brazil,Paris Saint-Germain,...,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3
3,193080,https://sofifa.com/player/193080/david-de-gea-...,De Gea,David De Gea Quintana,27,1990-11-07,193,76,Spain,Manchester United,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,27,1991-06-28,181,70,Belgium,Manchester City,...,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3
5,155862,https://sofifa.com/player/155862/sergio-ramos-...,Sergio Ramos,Sergio Ramos García,32,1986-03-30,184,82,Spain,Real Madrid,...,82+3,83+3,83+3,83+3,82+3,84+3,87+3,87+3,87+3,84+3
6,176580,https://sofifa.com/player/176580/luis-suarez/1...,L. Suárez,Luis Alberto Suárez Díaz,31,1987-01-24,182,86,Uruguay,FC Barcelona,...,69+5,68+5,68+5,68+5,69+5,66+5,63+5,63+5,63+5,66+5
7,177003,https://sofifa.com/player/177003/luka-modric/1...,L. Modrić,Luka Modrić,32,1985-09-09,172,66,Croatia,Real Madrid,...,82+3,81+3,81+3,81+3,82+3,79+3,71+3,71+3,71+3,79+3
8,183277,https://sofifa.com/player/183277/eden-hazard/1...,E. Hazard,Eden Hazard,27,1991-01-07,173,74,Belgium,Chelsea,...,66+3,63+3,63+3,63+3,66+3,60+3,49+3,49+3,49+3,60+3
9,200389,https://sofifa.com/player/200389/jan-oblak/19/...,J. Oblak,Jan Oblak,25,1993-01-07,188,87,Slovenia,Atlético Madrid,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
def clean(df):
  drop_cols=['sofifa_id','player_tags','player_traits','team_jersey_number','nation_jersey_number','loaned_from','joined','contract_valid_until','player_url','short_name','long_name','age','dob','value_eur','wage_eur','height_cm','weight_kg','nationality','release_clause_eur','club','ls','st','rs','lw','lf','cf','rf','rw','lam','cam','ram','lm','lcm','cm','rcm','rm','lwb','ldm','cdm','rdm','rwb','lb','lcb','cb','rcb','rb','work_rate','preferred_foot','real_face','body_type','nation_position','team_position']
  df_cleaned = df.drop(drop_cols, axis=1)
 
  forward = ["ST", "LW", "RW", "LF", "RF", "RS","LS", "CF"]
  midfielder = ["CM","RCM","LCM", "CDM","RDM","LDM", "CAM", "LAM", "RAM", "RM", "LM"]
  defender = ["CB", "RCB", "LCB", "LWB", "RWB", "LB", "RB"]
  
  for i in range(len(df_cleaned['player_positions'])):
    df_cleaned['player_positions'][i]=df_cleaned['player_positions'][i].replace(' ','').split(',')[0]
  
  df_cleaned.loc[df_cleaned["player_positions"] == "GK", "Position"] = 0
  df_cleaned.loc[df_cleaned["player_positions"].isin(defender), "Position"] = 1
  df_cleaned.loc[df_cleaned["player_positions"].isin(midfielder), "Position"] = 2
  df_cleaned.loc[df_cleaned["player_positions"].isin(forward), "Position"] = 3


  #one_hot_df = pd.get_dummies(df_cleaned['player_positions'], prefix='position')
  

  x=df_cleaned.drop(['player_positions','Position'],axis=1)
  #x=df_cleaned.drop(['player_positions'],axis=1)
  #y=one_hot_df.values

  y=df_cleaned['Position']

  cols = x.columns
  for col in cols:
    if(type(x[col][0]) == str):
      x[col] = [int(eval(i)) for i in x[col].values]
    else:
      x[col].fillna(np.mean(x[col]), inplace=True)
  
  return x,y

In [7]:
x , y =clean(df)

C:\Users\sahil\AppData\Local\Temp/ipykernel_15236/912580959.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['player_positions'][i]=df_cleaned['player_positions'][i].replace(' ','').split(',')[0]


In [8]:
x.shape

(17770, 51)

In [9]:
y

0        3.0
1        3.0
2        3.0
3        0.0
4        2.0
        ... 
17765    2.0
17766    3.0
17767    2.0
17768    1.0
17769    2.0
Name: Position, Length: 17770, dtype: float64

In [10]:
from keras.utils.np_utils import to_categorical
y_cat = to_categorical(y)

y_cat[:10]

array([[0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.]], dtype=float32)

In [11]:
x_train, x_test, y_train, y_test = train_test_split(x, y_cat,
                                                    test_size=0.2)

In [14]:
K.clear_session()
model = Sequential()
model.add(Dense(128, input_shape = (51,), activation = "relu"))
model.add(Dense(128, activation = "relu"))
model.add(Dense(128, activation = "relu"))
model.add(Dense(128, activation = "relu"))
model.add(Dense(128, activation = "relu"))
model.add(Dense(128, activation = "relu"))
model.add(Dense(128, activation = "relu"))
model.add(Dense(128, activation = "relu"))
model.add(Dense(128, activation = "relu"))
model.add(Dense(4, activation = "softmax"))
model.compile(tf.keras.optimizers.Adam(learning_rate = 0.001), "categorical_crossentropy", metrics = ["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               6656      
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_3 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_4 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_5 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_6 (Dense)              (None, 128)               1

In [16]:
model.fit(x_train, y_train, verbose=1, epochs=100)

Epoch 1/100
445/445 [==============================] - 1s 2ms/step - loss: 0.2592 - accuracy: 0.8920
Epoch 2/100
445/445 [==============================] - 1s 2ms/step - loss: 0.2585 - accuracy: 0.8915
Epoch 3/100
445/445 [==============================] - 1s 2ms/step - loss: 0.2587 - accuracy: 0.8894
Epoch 4/100
445/445 [==============================] - 1s 2ms/step - loss: 0.2572 - accuracy: 0.8928
Epoch 5/100
445/445 [==============================] - 1s 2ms/step - loss: 0.2585 - accuracy: 0.8918
Epoch 6/100
445/445 [==============================] - 1s 2ms/step - loss: 0.2531 - accuracy: 0.8939
Epoch 7/100
445/445 [==============================] - 1s 2ms/step - loss: 0.2496 - accuracy: 0.8950
Epoch 8/100
445/445 [==============================] - 1s 2ms/step - loss: 0.2479 - accuracy: 0.8955
Epoch 9/100
445/445 [==============================] - 1s 2ms/step - loss: 0.2490 - accuracy: 0.8938
Epoch 10/100
445/445 [==============================] - 1s 2ms/step - loss: 0.2449 - accura

445/445 [==============================] - 1s 2ms/step - loss: 0.2342 - accuracy: 0.9007
Epoch 82/100
445/445 [==============================] - 1s 2ms/step - loss: 0.3751 - accuracy: 0.8893
Epoch 83/100
445/445 [==============================] - 1s 2ms/step - loss: 0.2508 - accuracy: 0.8929
Epoch 84/100
445/445 [==============================] - 1s 2ms/step - loss: 0.2354 - accuracy: 0.9043
Epoch 85/100
445/445 [==============================] - 1s 2ms/step - loss: 0.2356 - accuracy: 0.8986
Epoch 86/100
445/445 [==============================] - 1s 2ms/step - loss: 0.3064 - accuracy: 0.8773
Epoch 87/100
445/445 [==============================] - 1s 2ms/step - loss: 0.2487 - accuracy: 0.8950
Epoch 88/100
445/445 [==============================] - 1s 2ms/step - loss: 0.2463 - accuracy: 0.8943
Epoch 89/100
445/445 [==============================] - 1s 2ms/step - loss: 0.2390 - accuracy: 0.8994
Epoch 90/100
445/445 [==============================] - 1s 2ms/step - loss: 0.2400 - accuracy: 

In [29]:
# To initialise our model we set some parameters commonly defined in an MLP design

# The number of nodes in the hidden layer
n_hidden = 128

# The number of input nodes
n_input = 51

# The number of nodes in the output layer
n_output = 4

In [30]:
input =tf.keras.Input(shape=(n_input,))

#define the dense layer, with number of hidden nodes as above and activation relu
layer1 = tf.keras.layers.Dense(n_hidden, activation='elu')(input)

layer2 = tf.keras.layers.Dense(n_hidden, activation='elu')(layer1)

layer3=tf.keras.layers.Dense(n_hidden, activation='elu')(layer2)

layer4=tf.keras.layers.Dense(n_hidden, activation='elu')(layer3)

layer5=tf.keras.layers.Dense(n_hidden, activation='elu')(layer4)

layer6=tf.keras.layers.Dense(n_hidden, activation='elu')(layer5)

layer7=tf.keras.layers.Dense(n_hidden, activation='elu')(layer6)
#define the output lyaer, with number of output nodes as defined above and softmax activation
output = tf.keras.layers.Dense(n_output, activation = 'softmax')(layer7)

#define the model by specifying the input and output to the model
model = tf.keras.Model(inputs= input, outputs= output, name="functional_api_nn")

In [31]:
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [32]:
history = model.fit(x_train, y_train, epochs = 100, verbose=1)

Epoch 1/100
445/445 [==============================] - 1s 2ms/step - loss: 0.4863 - accuracy: 0.8123
Epoch 2/100
445/445 [==============================] - 1s 2ms/step - loss: 0.3072 - accuracy: 0.8703
Epoch 3/100
445/445 [==============================] - 1s 2ms/step - loss: 0.2987 - accuracy: 0.8723
Epoch 4/100
445/445 [==============================] - 1s 2ms/step - loss: 0.2857 - accuracy: 0.8782
Epoch 5/100
445/445 [==============================] - 1s 2ms/step - loss: 0.3232 - accuracy: 0.8680
Epoch 6/100
445/445 [==============================] - 1s 2ms/step - loss: 0.2803 - accuracy: 0.8794
Epoch 7/100
445/445 [==============================] - 1s 2ms/step - loss: 0.2769 - accuracy: 0.8822
Epoch 8/100
445/445 [==============================] - 1s 2ms/step - loss: 0.2762 - accuracy: 0.8822
Epoch 9/100
445/445 [==============================] - 1s 2ms/step - loss: 0.2623 - accuracy: 0.8908
Epoch 10/100
445/445 [==============================] - 1s 2ms/step - loss: 0.2716 - accura

445/445 [==============================] - 1s 2ms/step - loss: 0.2380 - accuracy: 0.8986
Epoch 82/100
445/445 [==============================] - 1s 2ms/step - loss: 0.2377 - accuracy: 0.8991
Epoch 83/100
445/445 [==============================] - 1s 2ms/step - loss: 0.2371 - accuracy: 0.8981
Epoch 84/100
445/445 [==============================] - 1s 2ms/step - loss: 0.2522 - accuracy: 0.8960
Epoch 85/100
445/445 [==============================] - 1s 2ms/step - loss: 0.2400 - accuracy: 0.8980
Epoch 86/100
445/445 [==============================] - 1s 2ms/step - loss: 0.2350 - accuracy: 0.9007
Epoch 87/100
445/445 [==============================] - 1s 2ms/step - loss: 0.2347 - accuracy: 0.8993
Epoch 88/100
445/445 [==============================] - 1s 2ms/step - loss: 0.2999 - accuracy: 0.8814
Epoch 89/100
445/445 [==============================] - 1s 2ms/step - loss: 0.2482 - accuracy: 0.8944
Epoch 90/100
445/445 [==============================] - 1s 2ms/step - loss: 0.2428 - accuracy: 

In [ ]:
train_accuracy = model.evaluate(x_train,y_train)[1]
test_accuracy = model.evaluate(x_test,y_test)[1]

112/112 [==============================] - 0s 1ms/step - loss: 0.2794 - accuracy: 0.8852


In [ ]:
df_20=pd.read_csv('players_20.csv')

In [ ]:
x_20, y_20 =clean(df_20)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
from keras.utils.np_utils import to_categorical
y_cat_20 = to_categorical(y_20)

y_cat[:10]
model.evaluate(x_20,y_cat_20)

572/572 [==============================] - 1s 1ms/step - loss: 0.2565 - accuracy: 0.8944


[0.25648218393325806, 0.8943538665771484]